In [4]:
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt

from tensorflow.python.framework import ops

import os
import copy

from random import shuffle

In [44]:
def skip_connection(input_layer, filters, layer):
    x      = tf.layers.conv2d             (input_layer, filters=filters, kernel_size=(3, 3), padding='SAME', name="conv1_" + layer)
    output = tf.layers.batch_normalization(x, name="batch1_" + layer)
    output = tf.nn.relu                   (output, name="relu1_" + layer)
    
    output = tf.layers.conv2d             (output     , filters=filters, kernel_size=(3, 3), padding='SAME', name="conv2_" + layer)
    output = tf.layers.batch_normalization(output, name="batch2_" + layer)
    output = tf.nn.relu                   (output, name="relu2_" + layer)
    
    skip   = tf.layers.conv2d(x     ,filters=filters , kernel_size=(1, 1), strides=(1,1), padding='SAME', activation=tf.nn.relu, name="convskip_" + layer)
    skip   = tf.nn.relu(skip, name="reluskip_" + layer)
    
    return tf.keras.layers.concatenate([output, skip])

In [35]:
def start_model(model_input):
    output = tf.layers.conv2d(model_input, filters=64, kernel_size=(7,7), strides=(2,2))
    output = tf.layers.batch_normalization(output)
    output = tf.nn.relu(output)
    output = tf.layers.max_pooling2d(output, pool_size=(3,3) ,strides=(2,2))
    
    return output

In [49]:
ops.reset_default_graph()

model_input = tf.placeholder(dtype = tf.float32, shape = (None, 224, 224, 3))
random_input= np.random.rand(1, 224, 224, 3)

output = start_model(model_input)

#FIRST SET OF 64 FILTER CONVOLUTIONS
output = skip_connection(output, 64,  "1a")
output = skip_connection(output, 64,  "1b")
output = skip_connection(output, 64,  "1c")
output = skip_connection(output, 64,  "1d")
output = skip_connection(output, 64,  "1e")
output = skip_connection(output, 64,  "1f")

#SECOND SET OF 128 FILTER CONVOLUTIONS
output = skip_connection(output, 224, "2a")
output = skip_connection(output, 224, "2b")
output = skip_connection(output, 224, "2c")
output = skip_connection(output, 224, "2d")
output = skip_connection(output, 224, "2e")
output = skip_connection(output, 224, "2f")
output = skip_connection(output, 224, "2g")
output = skip_connection(output, 224, "2h")

#THIRD SET OF 256 FILTER CONVOLUTIONS
output = skip_connection(output, 224, "3a")
output = skip_connection(output, 224, "3b")
output = skip_connection(output, 224, "3c")
output = skip_connection(output, 224, "3d")
output = skip_connection(output, 224, "3e")
output = skip_connection(output, 224, "3f")
output = skip_connection(output, 224, "3g")
output = skip_connection(output, 224, "3h")
output = skip_connection(output, 224, "3i")
output = skip_connection(output, 224, "3j")
output = skip_connection(output, 224, "3k")
output = skip_connection(output, 224, "3l")

#FOURTH SET OF 512 FILTER CONVOLUTIONS
output = skip_connection(output, 224, "4a")
output = skip_connection(output, 224, "4b")
output = skip_connection(output, 224, "4c")
output = skip_connection(output, 224, "4d")
output = skip_connection(output, 224, "4e")
output = skip_connection(output, 224, "4f")

#AVERAGE POOL
output = tf.layers.average_pooling2d(output, pool_size=(2,2), strides=(2,2), name="avg_pool")

#FLATTEN AND PERFORM SOFTMAX
output = tf.layers.flatten(output)
output = tf.layers.dense(output, 1000)
init  = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    writer = tf.summary.FileWriter('./log_tmp/', sess.graph)
    
    skip = sess.run([output], feed_dict = { model_input : random_input})
    
print(skip[0].shape)

(1, 1000)


In [ ]:
def resnet_34(training_examples, learning_rate=1e-4, num_epochs=100, batch_size=32):
    ops.reset_default_graph()    
    model_input = tf.placeholder(dtype = tf.float32, shape = (None, 224, 224, 3))